# Clustering

## Load libraries

In [1]:
import sys
import os
import random
import numpy as np
from numba import jit, prange
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, average_precision_score
from sklearn.preprocessing import RobustScaler, LabelEncoder, StandardScaler, OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.utils import to_undirected, negative_sampling
import networkx as nx
from scipy.spatial import cKDTree
from scipy.special import expit
from typing import List, Dict
import time
import cProfile
import pstats
import io
import category_encoders as ce
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import copy
from torch_geometric.transforms import RandomNodeSplit
from collections import Counter
from category_encoders import BinaryEncoder
import cProfile
import pstats
import io



# Print versions of imported libraries
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {matplotlib.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"Torch version: {torch.__version__}")
print(f"Torch Geometric version: {torch_geometric.__version__}")
print(f"NetworkX version: {nx.__version__}")

if torch.cuda.is_available():
    device = torch.device("cuda")          # Current CUDA device
    print(f"Using {torch.cuda.get_device_name()} ({device})")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
else:
    print("CUDA is not available on this device.")

Python version: 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)]
NumPy version: 1.24.3
Pandas version: 2.0.1
Matplotlib version: 3.7.1
Scikit-learn version: 1.2.2
Torch version: 2.0.0+cu118
Torch Geometric version: 2.3.1
NetworkX version: 3.0
Using NVIDIA GeForce RTX 3060 Ti (cuda)
CUDA version: 11.8
Number of CUDA devices: 1


## Load data

In [2]:
dtypes = {
    'id': 'string',
    '#chrom': 'int64',
    'pos': 'int64',
    'ref': 'string',
    'alt': 'string',
    'rsids': 'string',
    'nearest_genes': 'string',
    'pval': 'float64',
    'mlogp': 'float64',
    'beta': 'float64',
    'sebeta': 'float64',
    'af_alt': 'float64',
    'af_alt_cases': 'float64',
    'af_alt_controls': 'float64',
    'finemapped': 'int64'
}

data = pd.read_csv('gwas-finemap.csv', dtype=dtypes)

# Assert column names
expected_columns = ['#chrom', 'pos', 'ref', 'alt', 'rsids', 'nearest_genes', 'pval', 'mlogp', 'beta',
                    'sebeta', 'af_alt', 'af_alt_cases', 'af_alt_controls', 'finemapped',
                    'id', 'trait']
assert set(data.columns) == set(expected_columns), "Unexpected columns in the data DataFrame."

# Assert data types
expected_dtypes = {
    'id': 'string',
    '#chrom': 'int64',
    'pos': 'int64',
    'ref': 'string',
    'alt': 'string',
    'rsids': 'string',
    'nearest_genes': 'string',
    'pval': 'float64',
    'mlogp': 'float64',
    'beta': 'float64',
    'sebeta': 'float64',
    'af_alt': 'float64',
    'af_alt_cases': 'float64',
    'af_alt_controls': 'float64',
    'finemapped': 'int64'
}

for col, expected_dtype in expected_dtypes.items():
    assert data[col].dtype == expected_dtype, f"Unexpected data type for column {col}."

In [3]:
# Check for total number of null values in each column
null_counts = data.isnull().sum()

print("Total number of null values in each column:")
print(null_counts)

Total number of null values in each column:
#chrom                   0
pos                      0
ref                      0
alt                      0
rsids              1366396
nearest_genes       727855
pval                     0
mlogp                    0
beta                     0
sebeta                   0
af_alt                   0
af_alt_cases             0
af_alt_controls          0
id                       0
finemapped               0
trait                    0
dtype: int64


## Data manipulation

In [4]:
data = data.sample(frac=0.05, random_state=42)

### Find nearest gene

In [5]:
data['nearest_genes'] = data['nearest_genes'].astype(str)

# Assert column 'nearest_genes' is a string
assert data['nearest_genes'].dtype == 'object', "Column 'nearest_genes' is not of string type."

# Get the length of the data before transformation
original_length = len(data)

# Extract the first gene name from the 'nearest_genes' column
data['nearest_genes'] = data['nearest_genes'].str.split(',').str[0]

# Reset index to have a standard index
data = data.reset_index(drop=True)

# Assert the length of the data remains the same
assert len(data) == original_length, "Length of the data has changed after transformation."

In [6]:
data = data.sample(frac=0.1, random_state=42)

## Spec

### Data

`data` Pandas DataFrame:

- `id`: This column represents the id of the variant in the following format: #chrom:pos:ref:alt (string).
- `#chrom`: This column represents the chromosome number where the genetic variant is located (int).
- `pos`: This is the position of the genetic variant on the chromosome (int: 1-200,000).
- `ref`: This column represents the reference allele (or variant) at the genomic position.
- `alt`: This is the alternate allele observed at this position.
- `rsids`: This stands for reference SNP cluster ID. It's a unique identifier for each variant used in the dbSNP database.
- `nearest_genes`: This column represents the gene which is nearest to the variant (string).
- `pval`: This represents the p-value, which is a statistical measure for the strength of evidence against the null hypothesis.
- `mlogp`: This represents the minus log of the p-value, commonly used in genomic studies.
- `beta`: The beta coefficient represents the effect size of the variant.
- `sebeta`: This is the standard error of the beta coefficient.
- `af_alt`: This is the allele frequency of the alternate variant in the general population (float: 0-1.
- `af_alt_cases`: This is the allele frequency of the alternate variant in the cases group (float: 0-1).
- `af_alt_controls`: This is the allele frequency of the alternate variant in the control group (float: 0-1).
- `finemapped`: This column represents whether the variant is included in the post-finemapped dataset (1) or not (0) (int).
- `trait`: This column represents the trait associated with the variant. In this dataset, it is the response to the drug paracetamol and NSAIDs.


### Nodes and Their Features

There is one type of node: SNP nodes.

- **SNP Nodes**: Each SNP Node is characterized by various features, including `id`, `nearest_genes`, `#chrom`, `pos`, `ref`, `alt`, `mlogp`, `beta`, `sebeta`,  `af_alt`, `af_alt_cases`, and `af_alt_controls` columns.

### Edges, Their Features, and Labels

Edges represent relationships between SNP nodes in the graph:

- For each pair of SNPs (row1 and row2) that exist on the same chromosome (`#chrom`), an edge is created if the absolute difference between their positions (`pos`) is less than or equal to 1,000,000 and greater than 1 (no loops).
-  Create a new edge attribute called `LD` for each edge. The value of `LD` is determined by the following formula:
     
```
weights = 1 * e^(-ln(2) / 100_000 * pos_diff_abs)
```

- The edge attributes (`edge_attr`) are:
   - `nearest_genes` 
   - `#chrom`
   - `pos`
   - `af_alt`, 
   - `af_alt_cases` 
   - `af_alt_controls`
   - `LD`

## Graph creation

## Clustering

In [7]:
from math import exp, log

edge_weight_cutoff = 1e-3  # set the cutoff value



def get_unique_snps(data: pd.DataFrame) -> dict:
    return {snp: idx for idx, snp in enumerate(data['id'].unique())}



def preprocess_snp_features(data: pd.DataFrame, snp_to_idx: dict) -> pd.DataFrame:
    cols_to_extract = ['id', 'nearest_genes', '#chrom', 'pos', 'ref', 'alt', 'mlogp', 'beta', 'sebeta', 'af_alt', 'af_alt_cases', 'af_alt_controls']
    snp_features = data.loc[data['id'].isin(snp_to_idx.keys()), cols_to_extract].set_index('id').sort_index()

    scaler = RobustScaler()

    categorical_cols = ['ref', 'alt', 'nearest_genes']
    count_encoder = ce.CountEncoder(cols=categorical_cols)
    snp_features = count_encoder.fit_transform(snp_features)

    numerical_cols = list(set(snp_features.columns) - set(categorical_cols))
    #snp_features[numerical_cols] = scaler.fit_transform(snp_features[numerical_cols])

    snp_features = snp_features.fillna(0)

    return snp_features



def preprocess_edges(data: pd.DataFrame, snp_to_idx: dict) -> torch.Tensor:
    scaler = RobustScaler()
    
    data = data.sort_values(by=['#chrom', 'pos'])
    data['snp_idx'] = data['id'].map(snp_to_idx)
    
    label_encoder = LabelEncoder()
    data['nearest_genes'] = label_encoder.fit_transform(data['nearest_genes'])
    
    edge_list = []
    edge_attributes = []

    for chrom, group in data.groupby('#chrom'):
        if group.empty:
            continue
        
        # We can use broadcasting to create a matrix of differences, this avoids the inner loop
        pos_diff = abs(group['pos'].values[:, None] - group['pos'].values)
        mask = (pos_diff > 1) & (pos_diff <= 1_000_000)

        filtered_group = group[mask]
        
        if filtered_group.empty:
            continue
        
        edge_list.extend([(idx, other_idx) for idx, others in enumerate(mask) for other_idx in others.nonzero()[0]])
        
        edge_attr = np.vstack([filtered_group['nearest_genes'].values,
                               filtered_group['#chrom'].values,
                               filtered_group['pos'].values,
                               filtered_group['af_alt'].values,
                               filtered_group['af_alt_cases'].values,
                               filtered_group['af_alt_controls'].values,
                               np.exp(-np.log(2) / 100_000 * pos_diff[mask])]).T
                               
        edge_attributes.append(edge_attr)

    edge_attributes = np.vstack(edge_attributes)
    edge_attributes = scaler.fit_transform(edge_attributes)
    
    return torch.tensor(edge_list, dtype=torch.long).t().contiguous(), torch.tensor(edge_attributes, dtype=torch.float)


def create_pytorch_graph(features: torch.Tensor, edges: torch.Tensor, edge_attr: torch.Tensor) -> Data:
    return Data(x=features, edge_index=edges, edge_attr=edge_attr)

def count_isolated_nodes(graph: Data) -> int:
    node_degrees = graph.edge_index[0].bincount(minlength=graph.num_nodes)
    isolated_nodes = (node_degrees == 0).sum().item()
    return isolated_nodes


# Create a profiler object
pr = cProfile.Profile()
pr.enable()

start_time = time.time()

snp_to_idx = get_unique_snps(data)
snp_features = preprocess_snp_features(data, snp_to_idx)
features = torch.tensor(snp_features.values, dtype=torch.float)

edges, edge_attr = preprocess_edges(data, snp_to_idx)
graph = create_pytorch_graph(features, edges, edge_attr)
graph.y = torch.tensor(data['finemapped'].values, dtype=torch.long)

print(f"Number of nodes: {graph.num_nodes}")
print(f"Number of edges: {graph.num_edges}")
print(f"Node feature dimension: {graph.num_node_features}")

isolated_nodes = count_isolated_nodes(graph)
print(f"Number of isolated nodes: {isolated_nodes}")

# Calculate elapsed time
elapsed_time = time.time() - start_time
print(f"Execution time: {elapsed_time} seconds")



pr.disable()
s = io.StringIO()
sortby = 'cumulative'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats(3)  # Only print the top 3 lines
print(s.getvalue())


Number of nodes: 100850
Number of edges: 7284022
Node feature dimension: 11
Number of isolated nodes: 92636
Execution time: 9.31654143333435 seconds
         3925058 function calls (3923667 primitive calls) in 9.168 seconds

   Ordered by: cumulative time
   List reduced from 1059 to 3 due to restriction <3>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       15    0.000    0.000    9.316    0.621 C:\Users\falty\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3472(run_code)
       15    0.000    0.000    9.316    0.621 {built-in method builtins.exec}
        1    2.215    2.215    7.858    7.858 C:\Users\falty\AppData\Local\Temp\ipykernel_21964\3294778173.py:31(preprocess_edges)





## PyTorch Geometric -> NetworkX

In [8]:
import networkx as nx

def pyg_to_nx(graph):
    # Initialize an empty graph
    G = nx.Graph()

    # Extract edge indices and attributes
    edge_index = graph.edge_index.t().tolist()
    edge_attr = graph.edge_attr.tolist()

    # Combine edge indices and attributes into a single list
    edges = [(src, dst, {"weight": attr[0]}) for (src, dst), attr in zip(edge_index, edge_attr)]  # Change here: assume the first element of attr is the weight
    
    # Add edges to the graph
    G.add_edges_from(edges)

    # Extract node features and create a dictionary with node index as keys
    node_features = {i: {f"feature_{j}": feat[j] for j in range(len(feat))} for i, feat in enumerate(graph.x.tolist())}  # Change here: convert list of lists to dict

    # Set node attributes
    nx.set_node_attributes(G, node_features)
    
    return G

nx_graph = pyg_to_nx(graph)

# Print the total number of nodes and edges in the graph
print(f"Number of nodes: {nx_graph.number_of_nodes()}")
print(f"Number of edges: {nx_graph.number_of_edges()}")

# Calculate average degree
degrees = [degree for _, degree in nx_graph.degree()]
average_degree = sum(degrees) / nx_graph.number_of_nodes()
print(f"Average degree: {average_degree}")


# Calculate and print the number of connected components
num_connected_components = nx.number_connected_components(nx_graph)
print(f"Number of connected components: {num_connected_components}")

# Find and print the largest connected component
largest_connected_component = max(nx.connected_components(nx_graph), key=len)
print(f"Largest connected component (number of nodes): {len(largest_connected_component)}")


Number of nodes: 8214
Number of edges: 403850
Average degree: 98.33211589968347
Number of connected components: 1
Largest connected component (number of nodes): 8214


### Louvain Algorithm

In [9]:
from community import community_louvain

# Use the Louvain method for community detection
partition = community_louvain.best_partition(nx_graph, weight='weight')

# print the number of communities and the nodes in each community
print(f"Number of communities: {len(set(partition.values()))}")
for i in set(partition.values()):
    print(f"Community {i}: {[nodes for nodes in partition.keys() if partition[nodes] == i]}")


ValueError: Bad node degree (-36.765336364507675)

### K-Means

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0)
clusters = kmeans.fit_predict(graph.x.numpy())

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Perform t-SNE dimensionality reduction
tsne = TSNE(n_components=2, random_state=0)
reduced_data = tsne.fit_transform(graph.x.numpy())

# Plot the data points
plt.figure(figsize=(10, 10))
scatter = plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=clusters)

# Add a color bar
colorbar = plt.colorbar(scatter)
plt.title('Cluster Visualization')
plt.show()


### Hierarchical Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering

hc = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
clusters = hc.fit_predict(graph.x.numpy())

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Perform t-SNE dimensionality reduction
tsne = TSNE(n_components=2, random_state=0)
reduced_data = tsne.fit_transform(graph.x.numpy())

# Plot the data points
plt.figure(figsize=(10, 10))
scatter = plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=clusters)

# Add a color bar
colorbar = plt.colorbar(scatter)
plt.title('Cluster Visualization')
plt.show()


### DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=3, min_samples=2)
clusters = dbscan.fit_predict(graph.x.numpy())

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Perform t-SNE dimensionality reduction
tsne = TSNE(n_components=2, random_state=0)
reduced_data = tsne.fit_transform(graph.x.numpy())

# Plot the data points
plt.figure(figsize=(10, 10))
scatter = plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=clusters)

# Add a color bar
colorbar = plt.colorbar(scatter)
plt.title('Cluster Visualization')
plt.show()


### grid_cluster

In [ ]:
from torch_cluster import grid_cluster
import matplotlib.colors as mcolors

# Assume pos_chrom is a tensor of shape [num_nodes, 2], 
# where the first column represents 'pos' and the second column represents '#chrom'
pos_chrom = torch.tensor(np.stack([snp_features['pos'].values, snp_features['#chrom'].values])).t()

# Define the size of the grid cells. You may want to adjust this according to your needs.
size = torch.tensor([2e7, 1])  # Creates a grid with cell size defined according to your data distribution

# Perform the grid clustering
cluster = grid_cluster(pos_chrom, size)

# Now, cluster is a tensor of size [num_nodes] where each element is the cluster index of the corresponding node
print(cluster)

# Get the 'pos' and '#chrom' data
x = pos_chrom[:, 0].numpy()  # extract 'pos'
y = pos_chrom[:, 1].numpy()  # extract '#chrom'

# Convert cluster tensor to numpy for use with matplotlib
cluster = cluster.numpy()

num_clusters = len(np.unique(cluster))
print("Number of clusters:", num_clusters)

# Create a colormap based on the number of unique clusters
cmap = mcolors.LinearSegmentedColormap.from_list('rainbow', plt.cm.rainbow(np.linspace(0, 1, len(np.unique(cluster)))))

plt.figure(figsize=(10, 8))
sc = plt.scatter(x, y, c=cluster, cmap=cmap, alpha=0.6)
plt.colorbar(sc, label='Cluster Index')
plt.xlabel('Position')
plt.ylabel('Chromosome Number')
plt.title('Grid Clustering of SNPs')

# Modify y-ticks to represent actual chromosome numbers
plt.yticks(range(1, int(y.max())+1))

plt.show()